In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject39_1526417507/1526417507.csv', header = None)
data = data.T
data2 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject39_1526591202/1526591202.csv', header = None)
data2 = data2.T
data3 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject42_1527280030/1527280030.csv', header = None)
data3 = data3.T
data4 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject43_1527806941/1527806941.csv', header = None)
data4 = data4.T
data5 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject54_1539288817/1539288817.csv', header = None)
data5 = data5.T
data6 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject55_1539459892/1539459892.csv', header = None)
data6 = data6.T
traindata = pd.concat([data,data2,data3,data4,data5,data6],ignore_index = True)

In [4]:
data = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject39_1526417507/breathrates.csv', header = None)
data = data.T
data = data.drop(0)
data2 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject39_1526591202/breathrates.csv', header = None)
data2 = data2.T
data2 = data2.drop(0)
data3 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject42_1527280030/breathrates.csv', header = None)
data3 = data3.T
data3 = data3.drop(0)
data4 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject43_1527806941/breathrates.csv', header = None)
data4 = data4.T
data4 = data4.drop(0)
data5 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject54_1539288817/breathrates.csv', header = None)
data5 = data5.T
data5 = data5.drop(0)
data6 = pd.read_csv('/content/drive/MyDrive/hackathon/ground_truth/Subject55_1539459892/breathrates.csv', header = None)
data6 = data6.T
data6 = data6.drop(0)
target_values = pd.concat([data,data2,data3,data4,data5,data6],ignore_index = True)

In [5]:
class LSTM(nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    self.lstm = nn.LSTM(input_size = 6780, hidden_size = 512, batch_first = True)
    self.linear1 = nn.Linear(512, 64)
    self.dropout = nn.Dropout(0.5)
    self.linear2 = nn.Linear(64, 1)
    self.relu = nn.ReLU()

  def forward(self, x):
    h_t, c_t = self.lstm(x)
    h_t = h_t.squeeze()
    res = self.linear1(h_t)
    res = self.relu(res)
    res = self.dropout(res)
    res = self.linear2(res)
    res = res.T
    return res

In [6]:
class BreathRateDataset(torch.utils.data.Dataset):
  def __init__(self, train_data, targets = None):
    self.train_data = train_data
    self.targets = targets

  def __len__(self):
    return len(self.train_data)

  def __getitem__(self, idx):
    if True:
      try:
        train, targ = self.train_data.loc[idx], self.targets.loc[idx]
      except:
        train, targ = self.train_data.iloc[idx], self.targets.iloc[idx]
      train = train.values.reshape((1,len(train)))
      targ = targ.values.reshape((1,len(targ)))
      train = train.astype(np.float32)
      targ = targ.astype(np.float32)
      train = torch.tensor(train)
      targ = torch.tensor(targ)
      train = train.view((1, 8, 6780))
      return train, targ

In [7]:
train_data, validation_data, train_target, validation_target = train_test_split(traindata, target_values, test_size=1/6, shuffle=False)

In [10]:
train_dataset = BreathRateDataset(train_data, train_target)
validation_dataset = BreathRateDataset(validation_data, validation_target)

In [11]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 1)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size = 1)

In [12]:
def train(model, lossfn, optimizer, scheduler, device):
    model.train()
    training_error = 0
    for t, (xb, yb) in enumerate(train_dataloader):
        xb = xb.to(device, dtype = torch.float32)
        yb = yb.to(device, dtype = torch.float32)
        xb = xb.squeeze(0)
        yb = yb.squeeze()
        predictions = model(xb)
        predictions = predictions.T
        predictions = predictions.to(device, dtype = torch.float32)
        loss = lossfn(predictions, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_error += torch.sqrt(torch.sum(torch.square(predictions.view(8)-yb))/8)
    print(f"Training RMSE: {training_error/5}")
    del loss,predictions
    return training_error

def validate(model, lossfn, optimizer, scheduler, device):
    validation_error = 0
    with torch.no_grad():
        model.eval()
        for v, (xv, yv) in enumerate(validation_dataloader):
            xv = xv.to(device, dtype = torch.float32)
            yv = yv.to(device, dtype = torch.int64)
            xv = xv.squeeze(0)
            yv = yv.squeeze()
            predictions = model(xv)
            predictions = predictions.T
            predictions = predictions.to(device, dtype = torch.float32)
            loss = lossfn(predictions, yv)
            validation_error += torch.sqrt(torch.sum(torch.square(predictions.view(8) - yv))/8) #rmse
    print(f"Validation RMSE: {validation_error/1}")
    return validation_error


In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [14]:
model = LSTM()
lossfn = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
scheduler = None
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, threshold=0.01, verbose = True)

In [15]:
epochs = 20
model.to(device)
for i in range(epochs):
    print(f"Epoch: {i+1}")
    training_error = train(model, lossfn, optimizer, scheduler, device)
    validation_error = validate(model, lossfn, optimizer, scheduler, device)

Epoch: 1


<ipython-input-12-b7ea3897c64c>:12: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = lossfn(predictions, yb)


Training RMSE: 15.203729629516602
Validation RMSE: 6.206967353820801
Epoch: 2


<ipython-input-12-b7ea3897c64c>:33: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = lossfn(predictions, yv)


Training RMSE: 12.555289268493652
Validation RMSE: 6.589913845062256
Epoch: 3
Training RMSE: 12.330035209655762
Validation RMSE: 7.694205284118652
Epoch: 4
Training RMSE: 13.20610523223877
Validation RMSE: 6.207226276397705
Epoch: 5
Training RMSE: 11.405604362487793
Validation RMSE: 5.378467559814453
Epoch: 6
Training RMSE: 9.994180679321289
Validation RMSE: 5.337657928466797
Epoch: 7
Training RMSE: 11.254755973815918
Validation RMSE: 5.668476104736328
Epoch: 8
Training RMSE: 11.355428695678711
Validation RMSE: 5.990256309509277
Epoch: 9
Training RMSE: 11.933667182922363
Validation RMSE: 6.081496238708496
Epoch: 10
Training RMSE: 11.211692810058594
Validation RMSE: 6.067468166351318
Epoch: 11
Training RMSE: 9.979409217834473
Validation RMSE: 6.347645282745361
Epoch: 12
Training RMSE: 11.349346160888672
Validation RMSE: 6.156043529510498
Epoch: 13
Training RMSE: 10.486631393432617
Validation RMSE: 6.219411849975586
Epoch: 14
Training RMSE: 10.653332710266113
Validation RMSE: 6.698429584

In [16]:
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
for i in range(epochs):
    print(f"Epoch: {i+1}")
    training_error = train(model, lossfn, optimizer, scheduler, device)
    validation_error = validate(model, lossfn, optimizer, scheduler, device)

Epoch: 1


<ipython-input-12-b7ea3897c64c>:12: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = lossfn(predictions, yb)


Training RMSE: 10.11539077758789
Validation RMSE: 6.985057830810547
Epoch: 2


<ipython-input-12-b7ea3897c64c>:33: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = lossfn(predictions, yv)


Training RMSE: 9.459980010986328
Validation RMSE: 7.1186370849609375
Epoch: 3
Training RMSE: 10.420019149780273
Validation RMSE: 7.137091636657715
Epoch: 4
Training RMSE: 10.237153053283691
Validation RMSE: 7.155633926391602
Epoch: 5
Training RMSE: 10.777729034423828
Validation RMSE: 7.110172271728516
Epoch: 6
Training RMSE: 8.964292526245117
Validation RMSE: 7.127218723297119
Epoch: 7
Training RMSE: 10.954889297485352
Validation RMSE: 7.162504196166992
Epoch: 8
Training RMSE: 10.349475860595703
Validation RMSE: 7.198596954345703
Epoch: 9
Training RMSE: 9.679279327392578
Validation RMSE: 7.238603115081787
Epoch: 10
Training RMSE: 10.71158504486084
Validation RMSE: 7.2389373779296875
Epoch: 11
Training RMSE: 10.777914047241211
Validation RMSE: 7.199800968170166
Epoch: 12
Training RMSE: 10.39391803741455
Validation RMSE: 7.198081970214844
Epoch: 13
Training RMSE: 11.160856246948242
Validation RMSE: 7.1399335861206055
Epoch: 14
Training RMSE: 9.751304626464844
Validation RMSE: 7.113490581

In [17]:
with torch.no_grad():
    model.eval()
    for v, (xv, yv) in enumerate(train_dataloader):
        xv = xv.to(device, dtype = torch.float32)
        yv = yv.to(device, dtype = torch.int64)
        xv = xv.squeeze(0)
        yv = yv.squeeze()
        predictions = model(xv)
        predictions = predictions.T
        predictions = predictions.to(device, dtype = torch.float32)
        print(predictions)
        print(yv)

tensor([[ 9.8981],
        [13.2756],
        [14.1891],
        [16.1445],
        [16.8532],
        [17.0338],
        [15.3431],
        [17.0449]])
tensor([16, 17, 16, 17, 12, 15, -1, -1])
tensor([[16.6691],
        [19.0042],
        [19.3621],
        [19.4114],
        [19.4181],
        [19.4190],
        [19.4191],
        [19.4191]])
tensor([36, 37, 37, 31, 35, 37, 35, 36])
tensor([[12.5017],
        [17.2311],
        [18.1451],
        [18.3422],
        [18.3715],
        [18.3755],
        [16.6376],
        [19.0731]])
tensor([11, 14, 17, 14, 14, 12, 11, 13])
tensor([[ 6.1134],
        [14.6679],
        [17.5494],
        [14.5577],
        [ 5.2670],
        [13.0045],
        [13.2321],
        [13.8895]])
tensor([11, 16, 16, 13, 13, -1, -1, -1])
tensor([[ 2.3638],
        [ 0.6290],
        [-0.2270],
        [ 0.3368],
        [-0.2947],
        [ 7.3519],
        [ 7.4743],
        [10.2661]])
tensor([10, -1, -1, -1, -1, -1, -1, -1])


In [18]:
with torch.no_grad():
    model.eval()
    for v, (xv, yv) in enumerate(validation_dataloader):
        xv = xv.to(device, dtype = torch.float32)
        yv = yv.to(device, dtype = torch.int64)
        xv = xv.squeeze(0)
        yv = yv.squeeze()
        predictions = model(xv)
        predictions = predictions.T
        predictions = predictions.to(device, dtype = torch.float32)
        print(predictions)
        print(yv)

tensor([[16.6691],
        [19.0042],
        [19.3621],
        [19.4114],
        [19.4181],
        [19.4190],
        [11.0789],
        [16.4294]])
tensor([15, 13, 15, 13, 16, 15, 12, -1])


In [19]:
predictions

tensor([[16.6691],
        [19.0042],
        [19.3621],
        [19.4114],
        [19.4181],
        [19.4190],
        [11.0789],
        [16.4294]])

In [20]:
target_values

,0,1,2,3,4,5,6,7
0,16.82,17.59,16.67,17.25,12.92,15.15,-1.00,-1.00
1,36.35,37.25,37.05,31.14,35.31,37.15,35.40,36.55
2,11.99,14.73,17.62,14.44,14.12,12.43,11.15,13.96
3,11.68,16.35,16.34,13.67,13.94,-1.00,-1.00,-1.00
4,10.23,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
5,15.30,13.26,15.30,13.11,16.01,15.17,12.85,-1.00


In [21]:
class TestDataset(torch.utils.data.Dataset):
  def __init__(self, train_data, targets = None):
    self.train_data = train_data

  def __len__(self):
    return len(self.train_data)

  def __getitem__(self, idx):
    try:
      train = self.train_data.loc[idx]
    except:
      train = self.train_data.iloc[idx]
    train = train.values.reshape((1,len(train)))
    train = train.astype(np.float32)
    train = torch.tensor(train)
    train = train.view((1, 8, 6780))
    return train

In [22]:
import os

df = pd.DataFrame()
addresses = []
for root, dir, files in os.walk('/content/drive/MyDrive/hackathon/dataset'):
  if files == []:
    pass
  else:
    path = os.path.join(root, files[0])
    data = pd.read_csv(path, header = None)
    addresses.append(root)
    data = data.T
    df = pd.concat([df,data], ignore_index = True)

In [23]:
test_dataset = TestDataset(df)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True)

In [24]:
with torch.no_grad():
        model.eval()
        for v, xv in enumerate(test_dataloader):
          xv = xv.to(device, dtype = torch.float32)
          xv = xv.squeeze(0)
          predictions = model(xv).view(8)
          predictions = predictions.detach().cpu().numpy()
          predictions = np.round(predictions, 2)
          path = addresses[v]
          time = int(path[-10:])
          timestamps = np.arange(time, time+240, 30)
          predictionsdf = pd.DataFrame([timestamps,predictions])
          predictionsdf = predictionsdf.transpose()

In [25]:
predictionsdf

,0,1
0,1.547585e+09,16.670000
1,1.547585e+09,18.190001
2,1.547585e+09,16.700001
3,1.547585e+09,19.379999
4,1.547585e+09,19.410000
5,1.547585e+09,19.420000
6,1.547585e+09,19.420000
7,1.547585e+09,19.420000


In [26]:
## The above dataframe matches the values of the tensors.